In [1]:
import dash 
from dash import dcc, html, Input, Output
import pandas as pd 
import plotly.graph_objects as go 
import sys 

sys.path.append('../')

import omdutils as omd

In [2]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='my-id', value='initial value', type='text'),
    html.Div(id='my-div', style={'border':'2px blue solid'})
])

@app.callback(Output(component_id='my-div', component_property='children'),
              Input(component_id='my-id', component_property='value'))

def get_message(input_value):
    return f'Input value was: "{input_value}"'

app.run(debug=True)

In [2]:
# df = pd.read_csv('../Data/gapminderDataFiveYear.csv')
db = '../SourceData/datasets.db'
sql = 'SELECT * FROM gapminderDataFiveYear'
# omd.to_sqlite(df, 'gapminderDataFiveYear', db)
df = omd.from_sqlite(sql, db)
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [3]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='year-picker', 
                 options=[{'label': i, 'value': i} for i in df['year'].unique()],
                 value=df['year'].min()),
    dcc.Graph(id='my-graph')
])

@app.callback(Output('my-graph', 'figure'),
              Input('year-picker', 'value'))
def update_plot(selected_year):

    filtered_df = df[df['year'] == selected_year]   
    data = [go.Scatter(x=filtered_df['gdpPercap'],
                       y=filtered_df['lifeExp'],
                       text=filtered_df['country'],
                       mode='markers',
                       marker=dict(size=15, opacity=0.6, line=dict(width=0.5, color='white')))]
    layout = go.Layout(title='Life Expectancy vs GDP Per Capita',
                       xaxis=dict(title='GDP Per Capita'),
                       yaxis=dict(title='Life Expectancy'),
                       hovermode='closest')
    return {'data': data, 'layout': layout}


app.run(debug=True)

In [15]:
data = [go.Bubble(x=df['year'],
                  y=df['gdpPercap'],
                  text=df['country'],
                  mode='markers',
                  marker=dict(size=df['pop']/1e6,
                                color=df['lifeExp'],
                                showscale=True))]

layout = go.Layout(title='Life Expectancy vs GDP Per Capita',
                   title_x=0.5,
                   xaxis=dict(title='Year'),
                   yaxis=dict(title='GDP Per Capita'))

fig = go.Figure(data=data, layout=layout)
fig.show()

AttributeError: module 'plotly.graph_objects' has no attribute 'Bubble'